In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fd378397730>)

In [2]:
pwd

'/Users/anseon-yeong/수업/2021-2/텍스트정보처리/프로젝트'

In [3]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [4]:
print('리뷰 개수 : ',len(train_data))

리뷰 개수 :  150000


In [5]:
train_data[:5]

,id,reviews,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
#데이터 중복 확인
train_data['reviews'].nunique(), train_data['label'].nunique

(146182,
 <bound method IndexOpsMixin.nunique of 0         0
 1         1
 2         0
 3         0
 4         1
          ..
 149995    0
 149996    1
 149997    0
 149998    1
 149999    0
 Name: label, Length: 150000, dtype: int64>)

In [7]:
#NULL 값을 가진 샘플 제거

train_data = train_data.dropna(how='any')
print(train_data.isnull().values.any())

False


In [8]:
print(len(train_data))

149995


In [9]:
train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data[:5]

,id,reviews,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [28]:
#train_data.to_csv('movie_reivews.txt',sep='\t',index=False)

In [10]:
train_data = train_data.drop(train_data.columns[0],axis='columns')
train_data

,reviews,label
0,아 더빙 진짜 짜증나네요 목소리,0
1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...
149995,인간이 문제지 소는 뭔죄인가,0
149996,평점이 너무 낮아서,1
149997,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [13]:
train_data['ratings'] = 0
train_data

,reviews,label,ratings
0,아 더빙 진짜 짜증나네요 목소리,0,0
1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,0
2,너무재밓었다그래서보는것을추천한다,0,0
3,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,0
...,...,...,...
149995,인간이 문제지 소는 뭔죄인가,0,0
149996,평점이 너무 낮아서,1,0
149997,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0,0
149998,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1,0


In [15]:
train_data.dtypes

reviews    object
label       int64
ratings     int64
dtype: object